# Chapter 13: Reflection and Iterative Refinement

**Scenario:** You are writing a **High-End Restaurant Review** for a luxury magazine. A generic AI response is too basic. You need a workflow where the AI writes a draft, **reflects** on its own quality, and then **refines** it to be perfect.

In this notebook, you will learn:
- 🔄 **The Reflection Pattern**: How to make the AI critique its own work.
- ✍️ **Iterative Refinement**: Using feedback to turn "Good" into "Great".
- 🎭 **Writer vs. Editor**: Using different personas to improve quality.

---

## 1. Setup

Standard setup.

In [1]:
# Install tools (if needed)
%pip install -q litellm python-dotenv

/Users/param/learn/learnwithparam/lwp-workshops/practical-prompt-engineering-workshop/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
from litellm import completion
import litellm
import logging

# Suppress noisy debug logs
litellm.suppress_debug_info = True
logging.getLogger("litellm").setLevel(logging.CRITICAL)

load_dotenv()
MODEL_NAME = os.getenv('DEFAULT_MODEL', 'gemini/gemini-2.5-flash')
print(f"Ready to refine work with: {MODEL_NAME}")

Ready to refine work with: openrouter/google/gemini-2.0-flash-001


In [3]:
def get_completion(prompt, system_prompt=None, temperature=0.0, max_tokens=1024):
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    
    response = completion(
        model=MODEL_NAME,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    
    return response.choices[0].message.content

## 2. The Reflection Loop

Instead of trying to get the perfect answer in one go, we use a loop:
1. **Generate**: Create the initial draft.
2. **Reflect**: Critique the draft (looking for style, tone, accuracy).
3. **Refine**: Apply the critique to create the final version.

Let's write a review for **"The Gilded Spoon"**.

In [4]:
restaurant_info = "The Gilded Spoon. High-end French. Great view. Beef Wellington was dry. Service was slow but polite. Expensive but worth it for the vibe."

draft_prompt = f"Write a one-paragraph restaurant review for 'The Gilded Spoon' based on these notes: {restaurant_info}. Tone should be sophisticated."
initial_draft = get_completion(draft_prompt)

print("--- INITIAL DRAFT ---")
print(initial_draft)

--- INITIAL DRAFT ---
The Gilded Spoon offers a decidedly opulent experience, transporting diners to a realm of classic French cuisine elevated by breathtaking panoramic views. While the Beef Wellington, regrettably, proved somewhat dry, the overall ambiance and meticulous attention to detail largely compensated for this singular misstep. Though the service, while undeniably polite, could benefit from a more expeditious pace, the restaurant's inherent charm and sophisticated atmosphere ultimately justify the elevated price point, making it a worthwhile indulgence for those seeking a memorable and refined evening.



### Step 2: Reflect (The Critique)

Now we act as a **Senior Food Editor** and find ways to improve the draft. We ask the model to be harsh and specific.

In [8]:
reflection_prompt = f"""
You are a Senior Food Editor at a luxury magazine.
Review the following draft restaurant review.

Critique it based on:
- VIVID SENSORY DETAILS: Does it describe tastes and smells?
- BALANCE: Does it address the dry beef fairly but firmly?
- VOCABULARY: Is it sophisticated or generic?

Draft:
{initial_draft}
"""

critique = get_completion(reflection_prompt, max_tokens=2048)
print("--- EDITOR'S CRITIQUE ---")
print(critique)

--- EDITOR'S CRITIQUE ---
Okay, here's my critique of the draft restaurant review for The Gilded Spoon, keeping in mind my role as a Senior Food Editor at a luxury magazine:

**Overall Impression:**

The draft is a good starting point, but it needs more *oomph* and specificity to truly capture the essence of a luxury dining experience. It feels a bit too general and relies on adjectives without providing enough concrete evidence to back them up. It's also a little too forgiving of the dry Beef Wellington, which is a significant flaw at this price point.

**Specific Critiques:**

*   **VIVID SENSORY DETAILS:** This is where the review needs the most work.

    *   **Tastes:** The review mentions "classic French cuisine" but doesn't describe any specific flavors. What herbs were used? Was the sauce rich and decadent? Did the dry Beef Wellington have any redeeming qualities in terms of flavor? We need to *taste* the food through the writing.
    *   **Smells:** What aromas filled the air 

### Step 3: Refine (The Final Version)

Finally, we give the original draft and the critique back to the AI and ask for a rewrite.

In [9]:
refinement_prompt = f"""
Rewrite the restaurant review below based on the Editor's critique.

Original Draft: {initial_draft}

Editor Critique: {critique}
"""

final_review = get_completion(refinement_prompt)
print("--- FINAL REFINED REVIEW ---")
print(final_review)

--- FINAL REFINED REVIEW ---
Here's the revised restaurant review, incorporating the editor's feedback:

Stepping into The Gilded Spoon is akin to entering a jewel box. Gleaming crystal chandeliers cast a warm, amber glow on plush velvet banquettes, while the hushed murmur of conversation blends seamlessly with the sophisticated strains of a live jazz trio. Floor-to-ceiling windows frame a breathtaking panorama of the city skyline, with the glittering spire of the Chrysler Building and the inky expanse of the East River providing a dramatic backdrop to the dining experience. The air itself is perfumed with the subtle aroma of truffle oil and the promise of rich, buttery sauces.

The menu, a testament to classic French technique, boasts dishes like escargots bathed in garlic-parsley butter, their earthy aroma a tantalizing prelude to the meal. However, the Beef Wellington, a dish that should be a symphony of textures and flavors, was unfortunately a discordant note. The pastry, while be

**Why wait?**
By splitting the task into steps, the model doesn't have to "juggle" both the creative writing and the self-critique at the same time. This leads to significantly higher quality output.

## 3. Exercise: The Marketing Polisher

**Goal:** Build a 2-step reflection loop to improve a **Marketing Catchphrase**.

**Requirements:**
1. **Draft**: Create a catchphrase for "A ultra-quiet electric toothbrush".
2. **Reflect**: Check if it's too technical or too boring.
3. **Refine**: Make it punchy and emotional.

**Product Notes:** Model X-Silent. Low vibration. 40,000 strokes. Fits in pocket.

In [7]:
# Write your reflection loop here

print("Starting Polisher...")

Starting Polisher...


## Summary

1.  **Reflection Pattern**: Break tasks into Generate → Reflect → Refine.
2.  **Personas Matter**: Use a "Senior Editor" persona for reflection to get better critiques.
3.  **Iterate to Excellence**: The second or third pass is almost always significantly better than the first.